In [15]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import unicodedata
import mysql.connector
import ipynb
from ipynb.fs.full.SqlQuery import brandAdd
from ipynb.fs.full.SqlQuery import categorieAdd
from ipynb.fs.full.SqlQuery import AddProduct
from ipynb.fs.full.SqlQuery import getCagId
from ipynb.fs.full.SqlQuery import getBrandId
from ipynb.fs.full.SqlQuery import AddProductAttribut
from ipynb.fs.full.SqlQuery import AddLocalStoreProduct
from ipynb.fs.full.SqlQuery import Addlocalstors

In [16]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
  database="scraping_data"
)

In [17]:
def zoom_links():
    zoom_list=[]
    web = requests.get('https://www.zoom.com.tn/').text
    parse = BeautifulSoup(web, 'lxml')
    head= parse.find_all('li', class_='parent')
    for h in head:
        try:
            link= h.find('a', class_='dropdown-toggle has-category').get('href')
            info= h.find('a', class_='dropdown-toggle has-category').text
        except:
            link = 'None'
            info ='None'
        if link != 'None':
            zoom_dict={
                'link': link,
                'info': info
            }
            zoom_list.append(zoom_dict)
    return zoom_list

In [18]:
def priceConv(price):
    c=''
    for p in price:
        if(p.isdigit()):
            c=c+p
    return int(c)

In [19]:
def zoom_details(link):
    web = requests.get(link)
    parse = BeautifulSoup(web.content, 'html.parser')
    list=[]
    try:
        name= parse.find('div', class_='pb-right-column col-xs-12 col-sm-4 col-md-4').h1.text
    except:
        name= 'None'
    try:
        uid= parse.find('span', class_='editable').text
    except:
        uid='Undifined'
    try:
        brand = parse.find('div', class_='marque_name').a.img.get('alt')
        brand_logo = parse.find('div', class_='marque_name').a.img.get('src')
    except:
        brand='Unknown'
        brand_logo='Undfined'
    try:
        des = parse.find('div', class_ ='rte align_justify').text.strip()
    except:
        des= 'None'
    try:
        img = parse.find('span', id ='view_full_size').img['src']
    except:
        img='None'
    try:
        price = parse.find('span', id ='our_price_display').text
        price = priceConv(price)
    except:
        price='Undfined'
    title,feautre='',''
    try:
        fich_tech= parse.find('table', class_='table-data-sheet').find_all('tr')
        for f in fich_tech:
            title=f.td.text
            feautre= [t.text for t in f.find_all('td')][1]
            fich_dec={
                'label':title.strip(),
                'data':feautre.strip()
            }
            list.append(fich_dec)
    except:
        list=[]
    details={
        'name':name,
        'uid':uid,
        'brand':brand,
        'brand_logo':brand_logo,
        'des':des,
        'img':img,
        'price':price,
        'fich_tech':list
    }
    return details

In [20]:
zoom_details('https://www.zoom.com.tn/informatique/7293-pc-portable-lenovo-ideapad-330-15ast-amd-a4-9125-4go-1to-81d600ddfe.html')

{'name': 'Pc Portable Lenovo IdeaPad 330-15AST - AMD A4-9125 - 4Go - 1To (81D600DDFE)',
 'uid': '81D600DDFE LAP-LN405',
 'brand': 'Lenovo',
 'brand_logo': 'https://www.zoom.com.tn/img/m/12-small_default.jpg',
 'des': 'Caractéristiques : Processeur: AMD A4-9125 up to 2.6 GHz, 2 Mo de cache / Mémoire : 4 Go DDR4 / Disque dur : 1To / Carte graphique : AMD Radeon R3 Graphics / Ecran : 15.6" HD\xa0 (1366 x 768px) Anti-glare / Lecteur de cartes / Graveur DVD / Wifi / Bluetooth\xa0 / USB 3.0 / USB 2.0 / Ethernet / HDMI / port jack / Free Dos / Garantie 1 an',
 'img': 'https://www.zoom.com.tn/39675-thickbox_default/pc-portable-lenovo-ideapad-330-15ast-amd-a4-9125-4go-1to-81d600ddfe.jpg',
 'price': 839000,
 'fich_tech': [{'label': 'Processeur', 'data': 'AMD A4-9125'},
  {'label': 'Mémoire', 'data': '4 Go'},
  {'label': 'Type Mémoire', 'data': 'DDR4'},
  {'label': 'Disque Dur', 'data': '1 To'},
  {'label': 'Carte Graphique', 'data': 'AMD Radeon R3 Graphics'},
  {'label': "Système d'exploitation"

In [21]:
def next_url(url, x):
    if(x == 1):
        url=url+'#/page-'+str(x)
    else:
        url=url[:-1]
        if(x>10):
            url=url[:-1]
        url=url+str(x)
    return url

In [22]:
def paginnation_lengh(info):
    if(info=='Informatique'):
        nb=4
    elif(info=='Téléphonie | Tablettes'):
        nb=4
    elif(info=='Stockage'):
        nb=2
    elif(info=='Impression | Scanner'):
        nb=2
    elif(info=='Photo & Vidéo'):
        nb=1
    elif(info=='Image & son'):
        nb=2
    elif(info=='Bureautique '):
        nb=4
    elif(info=='Electroménager'):
        nb=4
    elif(info=='Mode Beauté & Santé'):
        nb=1
    else:
        nb=1
        
    return nb

In [23]:
supCatId=0
list = zoom_links()
for l in list:
    info = l['info']
    link = l['link']
    categorieAdd(info, link, 22)
    supCatId=getCagId(info,22)
    web = requests.get(link)
    parse = BeautifulSoup(web.content, 'html.parser')
    leng=paginnation_lengh(info)
    for x in range(1, leng+1):
        link= next_url(link,x)
        print(link)
        web = requests.get(link)
        parse = BeautifulSoup(web.content, 'html.parser')
        card= parse.find_all('div', class_='ajax_block_product')
        for c in card:
            try:
                stock = c.find('span', class_='available-now').text
            except:
                stock='non disponible'
            try:
                det_link= c.find('a', class_='add_to_compare compare')['href']
            except:
                det_link='NO LINKS'
            if(det_link != 'NO LINKS'):
                det=zoom_details(det_link)
                brandAdd(det['brand'], det['brand_logo'], 22)
                supBrandId=getBrandId(det['brand'], 22)
                prodId=AddProduct(det['name'], det_link, det['uid'], det['des'], det['price'], stock, 22, supBrandId, supCatId, det['img'])
                if(det['fich_tech']):
                    AddProductAttribut(det['fich_tech'], prodId)

https://www.zoom.com.tn/695-informatique#/page-1
https://www.zoom.com.tn/695-informatique#/page-2
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplic

Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
https://www.zoom.com.tn/899-telephonie#/page-1
https://www.zoom.com.tn/899-telephonie#/page-2
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate 

Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicat

Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicat

Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicat

Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
https://www.zoom.com.tn/710-electromenager#/page-4
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Dupli